In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import zipfile
import re
import json
import collections
from lxml import etree
from xml.etree.ElementTree import XML
from random import shuffle
import pathlib
import itertools
import string
from difflib import SequenceMatcher

docxFileName = "quick_section4.docx"
docxZip = zipfile.ZipFile(docxFileName)
documentXML = docxZip.read('word/document.xml')
et = etree.XML(documentXML)
ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

### Using xpath to find main stations

In [2]:
mainstation_xpath = './w:r[not(preceding-sibling::w:r//w:t)][not(w:rPr/w:sz[@w:val=\"16\"])][..//w:b[not(@w:val=\"0\")]]/w:t[1]'
mainstation_xpath2 = './w:r[not(preceding-sibling::w:r//w:t)][not(w:rPr/w:sz[@w:val=\"16\"])][../w:pPr[w:pStyle]/w:rPr/w:b[@w:val=\"0\"]]/w:t[1]'
mainstation_xpath3 = './w:r[not(preceding-sibling::w:r//w:t)][not(w:rPr/w:sz[@w:val=\"16\"])][../w:pPr[w:pStyle[@w:val=\"Heading1\"]]/w:rPr]/w:t[1]'
mainstation_xpath4 = './w:r[not(preceding-sibling::w:r//w:t)][not(w:rPr/w:sz[@w:val=\"16\"])][../w:pPr[w:pStyle[@w:val=\"Heading2\"]]/w:rPr]/w:t[1]'
mainstation_xpath5 = './w:r[not(preceding-sibling::w:r//w:t)][not(w:rPr/w:sz[@w:val=\"16\"])][../w:pPr[w:pStyle[@w:val=\"Heading3\"]]/w:rPr]/w:t[1]'
mainstation_xpath6 = './w:r[not(preceding-sibling::w:r//w:t)][not(w:rPr/w:sz[@w:val=\"16\"])][../w:pPr[w:pStyle[@w:val=\"Heading4\"]]/w:rPr]/w:t[1]'
first_token_para_xpath = './w:r[//w:t]/w:t[1]'

stations = pd.DataFrame(columns=['station','type','description'])
        
def is_mainst(para, mainstation, initial_letter, counter):
    paraxp = para.xpath(first_token_para_xpath, namespaces=ns)
    mainxpath = ""
    if paraxp:
        
        # If text is capitalized (with exception for stations starting with "Mc")L
        if paraxp[0].text.isupper() or paraxp[0].text.startswith("Mc") and paraxp[0].text[2:].isupper():
            
            # See if xpath matches a mainstation xpath:
            mainxpath = para.xpath(mainstation_xpath, namespaces=ns)
            if not mainxpath:
                mainxpath = para.xpath(mainstation_xpath2, namespaces=ns)
            if not mainxpath:
                mainxpath = para.xpath(mainstation_xpath3, namespaces=ns)
            if not mainxpath:
                mainxpath = para.xpath(mainstation_xpath4, namespaces=ns)
            if not mainxpath:
                mainxpath = para.xpath(mainstation_xpath5, namespaces=ns)
            if not mainxpath:
                mainxpath = para.xpath(mainstation_xpath6, namespaces=ns)
                
            # Filter out station names of length 1, station names that start with initial
            # of previous main station (e.g. "Y BOOTHAM JUNCTION") or that start with an
            # open square bracket or parenthesis:
            if mainxpath and len(mainxpath[0].text.strip()) > 1 and not mainxpath[0].text.startswith(initial_letter + " ") and not mainxpath[0].text.startswith("[") and not mainxpath[0].text.startswith("("):
                counter += 1
                mainstation = mainxpath[0].text
                initial_letter = mainstation[0]
                return mainstation, initial_letter, counter
            else:
                return mainstation, initial_letter, counter
        else:
            return mainstation, initial_letter, counter
    else:
        return mainstation, initial_letter, counter

mainstation = ""
initial_letter = ""
lowerstation = ""
dText = dict()
counter = 0
for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
    text = para.xpath('./w:r/w:t', namespaces=ns)
    description = " ".join([t.text for t in text])
    mainstation, initial_letter, counter = is_mainst(para, mainstation, initial_letter, counter)
    print(mainstation)
    description = description.lstrip('\x01').strip()
    if description:
        if (counter, mainstation) in dText:
            dText[(counter, mainstation)].append(description)
        else:
            description = re.sub('^(' + re.escape(mainstation) + ')', '\1', description).lstrip('\x01').strip()
            description = re.sub(r" +", " ", description).lstrip('\x01').strip()
            if description:
                dText[(counter, mainstation)] = [description]

ABBEY
ABBEY
ABBEY
ABBEY
ABBEY
ABBEY & WEST DEREHAM 
ABBEY FOREGATE 
ABBEY HOUSES 
ABBEY MILLS 
ABBEY OF DEER
ABBEY ROAD
ABBEY WOOD
ABBEY WOOD
ABBEY WOOD
ABBEYDORE 
ABBEYHILL 
ABBEYHOLME
ABBOTS RIPTON
ABBOTS WOOD JUNCTION
ABBOTSBURY
ABBOTSCLIFFE BOX
ABBOTSFORD FERRY
ABBOTSHAM ROAD 
ABDIE
ABER
ABER
ABER
ABERAMAN
ABERAMAN
ABERAMAN
ABERANGELL 
ABERAVON
ABERAVON
ABERAVON
ABERAYRON
ABERBARGOED
ABERBARGOED
ABERBARGOED
ABERBARGOED
ABERBARGOED
ABERBEEG
ABERBRAN 
ABERCAIRNY 
ABERCAMLAIS
ABERCANAID 
ABERCARN 
ABERCHALDER 
ABERCORN
ABERCRAVE 
ABERCWMBOI 
ABERCYNON
ABERCYNON
ABERCYNON
ABERCYNON
ABERCYNON
ABERCYNON
ABERDARE
ABERDARE
ABERDARE
ABERDARE
ABERDARE
ABERDARE JUNCTION 
ABERDEEN
ABERDEEN
ABERDEEN
ABERDEEN
ABERDEEN
ABERDEEN
ABERDEEN
ABERDEEN
ABERDEEN
ABERDEEN
ABERDEEN
ABERDEEN
ABERDEEN
ABERDEEN
ABERDEEN
ABERDERFYN 
ABERDOUR 
ABERDOVEY
ABERDOVEY
ABERDOVEY
ABERDYLAIS 
ABEREDW
ABERERCH 
ABERFAN 
ABERFELDY 
ABERFFRWD
ABERFORD
ABERFOYLE 
ABERGAVENNY
ABERGAVENNY
ABERGAVENNY
ABERGAVENNY
ABERGAVENNY


BAWDRIP
BAWTRY
BAXENDEN
BAY HORSE
BAYFORD
BAYNARDS
BAYSWATER
BEACH ROAD
BEACH STREET
BEACHLEY JUNCTION
BEACONSFIELD
BEACONSFIELD
BEACONSFIELD
BEAG FAIR SIDING
BEAL 
BEALINGS
BEAM BRIDGE
BEAM BRIDGE
BEAMISH
BEANACRE
BEARLEY
BEARPARK
BEARSDEN
BEARSTED
BEASDALE
BEASDALE
BEATTOCK
BEATTOCK
BEATTOCK
BEAUCHIEF
BEAUFORT
BEAUFORT
BEAUFORT
BEAULIEU ROAD
BEAULIEU ROAD
BEAULIEU ROAD
BEAULY
BEAULY
BEAULY
BEAUMONTS
BEAVERS HILL
BEBINGTON
BEBSIDE
BECCLES
BECKENHAM
BECKENHAM
BECKENHAM
BECKENHAM
BECKENHAM
BECKERMET
BECKERMET
BECKERMET
BECKFOOT
BECKFORD
BECKHOLE(S)
BECKHOLE(S)
BECKHOLE(S)
BECKINGHAM
BECKTON
BECKTON
BECKTON
BECKTON
BECKTON
BECONTREE
BECONTREE
BEDALE
BEDDAU
BEDDAU
BEDDGELERT
BEDDINGTON LANE
BEDE
BEDFORD
BEDFORD
BEDFORD
BEDFORD
BEDFORD
BEDFORD
BEDFORD
BEDFORD
BEDFORD
BEDFORD
BEDFORD LEIGH
BEDHAMPTON
BEDLAY 
BEDLINGTON
BEDLINOG
BEDLINOG
BEDLINOG
BEDLINOG
BEDMINSTER
BEDMINSTER
BEDMINSTER
BEDMINSTER
BEDMINSTER
BEDWAS
BEDWELLTY PITS 
BEDWELLTY PITS 
BEDWORTH
BEDWORTH
BEDWORTH
BEDWYN
BEECHBURN


BROOKHAY
BROOKLAND
BROOKLANDS
BROOKMANS PARK
BROOKSBY
BROOKWOOD
BROOKWOOD
BROOKWOOD
BROOM CRAIG
BROOM JUNCTION
BROOME
BROOMFIELD
BROOMFLEET
BROOMHILL
BROOMHILL
BROOMHILL
BROOMHILL
BROOMHOPE
BROOMHOUSE
BROOMIEKNOWE
BROOMIELAW
BROOMIELAW HARBOUR
BROOMLEE
BRORA
BROTHERTON
BROTTON
BROUGH
BROUGHTON
BROUGHTON
BROUGHTON & BRETTON
BROUGHTON ASTLEY
BROUGHTON CROSS
BROUGHTON GIFFORD
BROUGHTON LANE
BROUGHTON SKEOG
BROUGHTON-IN-FURNESS
BROUGHTY FERRY
BROUGHTY FERRY
BROUGHTY FERRY
BROUGHTY FERRY
BROWNDOWN
BROWNHILLS
BROWNHILLS
BROWNHILLS
BROXBOURNE
BROXBOURNE
BROXBURN
BROXBURN
BROXBURN
BROXTON
BRUCE GROVE
BRUCKLAY
BRUCKLAY
BRUCKLAY
BRUNDALL
BRUNDALL
BRUNDALL
BRUNDALL
BRUNSTANE
BRUNSWICK
BRUNSWICK
BRUNSWICK
BRUNSWICK
BRUTON
BRYMBO
BRYMBO
BRYMBO
BRYMBO
BRYMBO
BRYN
BRYN
BRYN
BRYN TEIFY
BRYN-Y-GWYNON
BRYNAMMAN
BRYNAMMAN
BRYNAMMAN
BRYNAMMAN
BRYNCELYNOG
BRYNDERWEN
BRYNGLAS
BRYNGWYN
BRYNGWYN
BRYNKIR
BRYNMAWR
BRYNMAWR
BRYNMENYN
BRYNMILL
BRYNTEG COLLIERY
BUBWITH
BUBWITH
BUBWITH
BUCHANAN STREET
BUCHANAN STRE

CLEVEDON
CLEVEDON
CLEVEDON
CLEVEDON
CLEVEDON
CLEVEDON ROAD
CLEVELAND BRIDGE 
CLEVELAND PORT
CLEVELEYS
CLIBURN
CLIDDESDEN
CLIFF COMMON
CLIFF COMMON
CLIFF COMMON
CLIFFE
CLIFFE
CLIFFE PARK
CLIFFE VALE
CLIFFORD
CLIFFORD
CLIFTON
CLIFTON
CLIFTON
CLIFTON
CLIFTON
CLIFTON
CLIFTON
CLIFTON
CLIFTON
CLIFTON
CLIFTON 
CLIFTON MILL
CLIFTON ROAD BRIGHOUSE
CLIFTON-ON-TRENT
CLIFTONVILLE
CLIPPENS
CLIPSTON & OXENDON
CLIPSTONE CAMP
CLIPSTONE COLLIERY 
CLITHEROE
CLITHEROE
CLITHEROE
CLOCK FACE
CLOCK FACE
CLOCK FACE
CLOCK HOUSE
CLOCKSBRIGGS
CLOGWYN
CLOSEBURN
CLOUGH FOLD
CLOUGHTON
CLOVENFORDS
CLOWNE
CLOWNE
CLOWNE
CLOWNE
CLOY
CLUBMOOR
CLUNDERWEN
CLUNES
CLUTTON
CLYDACH
CLYDACH COURT
CLYDACH VALE
CLYDACH-ON-TAWE
CLYDACH-ON-TAWE
CLYDACH-ON-TAWE
CLYDEBANK
CLYDEBANK
CLYDEBANK
CLYDEBANK
CLYDEBANK
CLYNDERWEN
CLYNE
CLYST ST MARY & DIGBY
COALBROOKDALE
COALBROOKDALE
COALBROOKDALE
COALBROOKDALE
COALBURN
COALEY
COALEY
COALPIT HEATH
COALPORT
COALPORT
COALPORT
COALVILLE
COALVILLE
COALVILLE
COALVILLE
COALYBURN
COANWOOD
COATBRI

DONIBRISTLE
DONIFORD
DONINGTON ROAD
DONINGTON-ON-BAIN
DONISTHORPE
DONNINGTON
DONYATT
DORCHESTER
DORCHESTER
DORCHESTER
DORE & TOTLEY 
DORKING
DORKING
DORKING
DORKING
DORKING
DORMANS
DORNOCH
DORNOCK
DORRIDGE
DORRINGTON
DORSTONE
DORTON
DORTON
DORTON
DOSELEY
DOUBLEBOIS
DOUGLAS
DOUGLAS WEST
DOULTING
DOUNE
DOUSLAND
DOVE BANK
DOVE HOLES
DOVECLIFFE
DOVENBY
DOVER
DOVER
DOVER
DOVER
DOVER
DOVER
DOVER
SE 
SE 
LCD 
LCD 
LCD 
LCD 
LCD 
DOVER STREET 
DOVERCOURT
DOVEY JUNCTION
DOVEY JUNCTION
DOWDESWELL
DOWLAIS
DOWLAIS
DOWLAIS
DOWLAIS
DOWLAIS
DOWLAIS
DOWLAIS
DOWLAIS
DOWLAIS TOP
DOWLAIS TOP
DOWLAIS TOP
DOWLOW
DOWN STREET
DOWN STREET
DOWNFIELD CROSSING
DOWNHAM MARKET
DOWNHOLLAND
DOWNTON
DR DAYS BRIDGE
DRAX
DRAX
DRAX
DRAYCOTT
DRAYCOTT & BREASTON
DRAYCOTT CAMP
DRAYTON
DRAYTON
DRAYTON GREEN
DRAYTON PARK
DRAYTON PARK
DRAYTON PARK
DREGHORN
DREM
DRIFFIELD
DRIFFIELD
DRIFFIELD
DRIGG
DRIGHLINGTON
DROITWICH
DROITWICH
DROITWICH
DROMORE
DRONFIELD
DRONFIELD
DRONFIELD
DRONGAN
DRONLEY
DROXFORD
DROYLSDEN 
DRUM
DRUMBURGH

...

PINEWOOD
PINEWOOD
PINGED
PINHOE
PINHOE
PINHOE
PINKHILL
PINMORE
PINNER
PINNER
PINNER
PINWHERRY
PINXTON
PINXTON
PINXTON
PINXTON
PIPE GATE
PIRTON
PITCAPLE
PITCROCKNIE
PITFODELS
PITFOUR CURLING
PITLOCHRY
PITLURG
PITMEDDEN
PITMUIES MILL
PITSEA
PITSFORD & BRAMPTON
PITTENWEEM
PITTENZIE
PITTINGTON
PITT’S HEAD
PITTS HILL
PLAIDY
PLAINS
PLAISTOW
PLAISTOW
PLANK LANE
PLANK LANE
PLANK LANE
PLANTATION
PLAS
PLAS MARL
PLAS POWER
PLAS POWER
PLAS POWER
PLAS-Y-COURT
PLAS-Y-NANT
PLASHETTS
PLATT BRIDGE
PLATT BRIDGE
PLATT BRIDGE
PLAWSWORTH
PLEALEY ROAD
PLEAN
PLEASINGTON
PLEASLEY
PLEASLEY
PLEASLEY
PLEASURE BEACH
PLECK
PLENMELLER
PLESSEY
PLESSEY
PLEX MOSS LANE
PLOCKTON
PLOCKTON
PLODDER LANE
PLOWDEN
PLUCKLEY
PLUCKLEY
PLUM(B)LEY
PLUM(B)LEY
PLUM(B)LEY
PLUMPTON
PLUMPTON
PLUMSTEAD
PLUMSTEAD
PLUMTREE
PLYM BRIDGE
PLYMOUTH
PLYMOUTH
PLYMOUTH
PLYMOUTH
PLYMOUTH
PLYMOUTH
PLYMOUTH
PLYMOUTH
PLYMOUTH
PLYMOUTH
PLYMOUTH
PLYMOUTH
PLYMOUTH
PLYMPTON
PLYMSTOCK
POCHIN PITS
POCHIN PITS
POCHIN PITS
POCKLINGTON
POINT PLEASANT
POISON C

RYDE
RYDE
RYDER BROW
RYDERS HAYS
RYE
RYE
RYE
RYE HILL & BURSTWICK
RYE HOUSE
RYEFORD
RYELAND
RYELAND
RYHILL
RYHILL
RYHILL
RYHOPE
RYHOPE
RYHOPE
RYHOPE
RYLSTONE
RYSTON
RYTON
RYTON
RYTON
S & R COLLIERY
SADDLEWORTH
SAFFRON WALDEN
SAFFRON WALDEN
SAFFRON WALDEN
ST AGNES
ST ALBANS
ST ALBANS
ST ALBANS
ST ALBANS
ST ANDREW(S)/ST A ROAD
ST ANDREWS
ST ANDREWS ROAD
ST ANNES PARK
ST ANNES-ON-THE-SEA
ST ANNS ROAD
ST ANNS WELL
ST ANTHONYS
ST ASAPH
ST ATHAN
ST ATHAN ROAD
ST AUSTELL
ST AUSTELL
ST AUSTELL
ST BEES
ST BEES
ST BEES
ST BEES
ST BLAZEY
ST BOSWELLS
ST BOTOLPHS
ST BRIAVELS
ST BRIDES CROSSING
ST BUDEAUX
ST BUDEAUX
ST BUDEAUX
ST BUDEAUX
ST CLEARS
ST COLUMB ROAD
ST COMBS
ST CYRES
ST CYRUS
ST DENYS
ST DEVEREUX
ST DUNSTANS
ST ENOCH
ST ENOCH
ST ENOCH
ST ERTH
ST ERTH
ST FAGANS
ST FAGANS
ST FAGANS
ST FILLANS
ST FORT
ST FORT
ST GABRIELS
ST GABRIELS
ST GEORGES CROSS
ST GERMANS
ST GERMANS
ST GERMANS
ST GERMANS
ST HARMONS
ST HELENS
ST HELENS
ST HELENS
ST HELENS
ST HELENS
ST HELENS
ST HELENS
ST HELENS
ST HELE

STALBRIDGE
STALEY & MILLBROOK
STALHAM
STALLINGBOROUGH
STALYBRIDGE
STALYBRIDGE
STALYBRIDGE
STALYBRIDGE
STAMFORD
STAMFORD
STAMFORD
STAMFORD
STAMFORD
STAMFORD
STAMFORD BRIDGE
STAMFORD BROOK
STAMFORD HILL
STAMMERHAM
STAMMERHAM
STAMMERHAM
STANBRIDGEFORD
STANBRIDGEFORD
STANBRIDGEFORD
STANDISH
STANDON
STANDON BRIDGE
STANE STREET
STANELY
STANFORD HALL
STANFORD ROAD
STANFORD-LE-HOPE
STANHOE
STANHOPE
STANHOPE
STANHOPE
STANLEY
STANLEY
STANLEY
STANLEY BRIDGE
STANLOW & THORNTON
STANMORE
STANMORE
STANMORE
STANNER
STANNERGATE
STANNERGATE
STANNINGLEY
STANNINGTON
STANSFIELD HALL
STANSTED
STANSTED
STANSTED
STANTON
STANTON GATE
STANWARDINE
STANWELL MOOR
STAPLE
STAPLE EDGE
STAPLE HILL
STAPLEFORD
STAPLEFORD & SANDIACRE
STAPLEHURST
STAPLETON
STAPLETON ROAD
STAR CROSSING
STAR LANE
STARBECK
STARCROSS
STARSTON
STAVELEY
STAVELEY
STAVELEY
STAVELEY
STAVELEY
STAVELEY
STAVELEY
STAVERTON
STAVERTON
STAWARD
STEAM FERRY CROSSING 
STEAM MILLS CROSSING
STECHFORD
STEELE ROAD
STEENS BRIDGE
STEEPLEHOUSE
STEER POINT
STEETON 

TURNHAM GREEN
TURNHOUSE
TURNPIKE LANE
TURRIFF
TURTON & EDGWORTH
TURVEY
TUTBURY
TUTBURY
TUTBURY
TUTSHILL
TUXFORD
TUXFORD
TUXFORD
TUXFORD
TUXFORD
TWECHAR
TWEEDBANK
TWEEDMOUTH
TWENTY
TWERTON-ON-AVON
TWICKENHAM
TWIZELL
TWYFORD
TWYFORD
TWYFORD ABBEY
TWYFORD BRIDGE
TWYMYN BRIDGE
TWYWELL
TY COCH
TY CROES
TY FYSTON
TY GLAS
TYCOCH
TYDD
TYDDENHENGOED
TYDDYN BRIDGE
TYDDYNGWYN
TYDEE/TYDU
TYGWYN
TYLACOCH
TYLDESLEY
TYLER HILL
TYLERS ARMS
TYLLWYN
TYLORSTOWN
TYLWCH
TYN-Y-FFRAM
TYNDRUM
TYNDRUM
TYNDRUM
TYNE COMMISSION QUAY
TYNE DOCK
TYNE DOCK 
TYNE DOCK 
TYNEHEAD
TYNEMOUTH
TYNEMOUTH
TYNEMOUTH
TYNEMOUTH
TYNEMOUTH
TYNEMOUTH
TYNEMOUTH
TYNEMOUTH
TYNEMOUTH
TYNEMOUTH
TYNEWYDD
TYNYCWM
TYNYFFREM 
TYNYLLWYN
TYSELEY
TYSELEY
TYSELEY
TYTHERINGTON
TYTHERINGTON
TYTHERINGTON
TYWITH
TYWYN
TYWYN
TYWYN
TYWYN
TYWYN
UCKFIELD
UDDINGSTON
UDDINGSTON
UDDINGSTON
UDDINGSTON
UDDINGSTON
UDNY
UFFCULME
UFFINGTON
UFFINGTON & BARNACK
UFFORD BRIDGE
ULBSTER
ULCEBY
ULCEBY
ULCEBY
ULLESKELF
ULLESTHORPE
ULLOCK
ULLOCK
ULVERSTON
ULVERSTON
ULV

WINCANTON
WINCHBURGH
WINCHCOMBE
WINCHELSEA
WINCHESTER
WINCHESTER
WINCHESTER
WINCHESTER
WINCHFIELD
WINCHMORE HILL
WINCOBANK
WINDER
WINDERMERE
WINDERMERE
WINDERMERE
WINDERMERE
WINDMILL END
WINDMILL HILL
WINDSOR & ETON
WINDSOR & ETON
WINDSOR & ETON
WINDSOR & ETON
WINDSOR BRIDGE
WINDSOR COLLIERY
WINDYHILL
WINESTEAD
WINGATE 
WINGFIELD
WINGFIELD VILLAS
WINGHAM
WINGHAM
WINGHAM
WINGHAM
WINGHAM
WINKHILL
WINMARLEIGH
WINNERSH
WINNERSH
WINNERSH
WINSCOMBE SOMERSET
WINSFORD
WINSFORD
WINSFORD
WINSLOW
WINSLOW
WINSLOW
WINSON GREEN
WINSON GREEN
WINSON GREEN
WINSON GREEN
WINSTON
WINTERBOURNE
WINTERINGHAM
WINTERINGHAM
WINTERSETT & RYHILL
WINTERTON & THEALBY
WINTHORPE
WINTON
WINTON PIER
WINWICK QUAY
WINWICK QUAY
WIRKSWORTH
WISBECH
WISBECH
WISBECH
WISBECH
WISBECH
WISBECH
WISBECH
WISHAW
WISHAW
WISHAW
WISHFORD
WISHING WELL HALT 
WISTANSTOW
WISTASTON ROAD
WISTOW
WITHAM
WITHAM SOMERSET
WITHCALL
WITHERENDEN
WITHERNSEA
WITHERNSEA
WITHINGTON
WITHINGTON & WEST DIDSBURY
WITHINGTON & WEST DIDSBURY
WITHINGTON GLOS
WIT

### Indexing main stations

In [3]:
dStations = collections.OrderedDict(dText)

indices = []
stations = []
descriptions = []
for k in dStations:
    indices.append(k[0])
    stations.append(k[1])
    descriptions.append(dStations[k])

stationdf = pd.DataFrame(columns=["Index", "Station", "Description"])
stationdf["Index"] = indices
stationdf["Station"] = stations
stationdf["Description"] = descriptions
stationdf = stationdf.set_index("Index")

stationdf.to_pickle('quick_processed.pkl')

In [4]:
stationdf.head()

,Station,Description
Index,,
1,ABBEY,[A TOWN [NB] op 3 September 1856** as A; TOWN ...
2,ABBEY & WEST DEREHAM,[[GE] op 1 August 1882 (Thetford & Watton Time...
3,ABBEY FOREGATE,[– see SHREWSBURY.]
4,ABBEY HOUSES,[see BEAUCHIEF .]
5,ABBEY MILLS,[pumping station [private line south of West H...


### Using regex to find substations

In [5]:
# From Quicks intro:
companies = {"AN Jt":"Ashby & Nuneaton Joint.",
    "ANSW":"Alexandra (Newport & South Wales) Docks & Railway.",
    "Ax Jt":"Axholme Joint.",
    "Bak":"Bakerloo Line.",
    "BC":"Bishops Castle.",
    "BE":"Bristol & Exeter.",
    "BG":"Birmingham & Gloucester.",
    "BLCJ":"Birkenhead, Lancashire & Cheshire Junction.",
    "BM":"Brecon & Merthyr.",
    "BPGV":"Burry Port & Gwendraeth Valley.",
    "BR":"British Railways (whilst nationalised).",
    "BT":"Blyth & Tyne.",
    "BWA":"Bideford, Westward Ho! & Appledore. ",
    "Cal":"Caledonian.",
    "Cam":"Cambrian.",
    "Camp":"Campbeltown & Machrihanish Light.",
    "CE":"Clifton Extension Joint.",
    "Cen":"Central Line.",
    "CGU":"City of Glasgow Union.",
    "CHP":"Cromford & High Peak.",
    "CKP":"Cockermouth, Keswick & Penrith.",
    "CLC":"Cheshire Lines Committee. ",
    "CMDP":"Cleobury Mortimer & Ditton Priors Light.",
    "CO Jt":"Croydon & Oxted Joint.",
    "Croydon":"Croydon Tramlink.",
    "CVH":"Colne Valley & Halstead.",
    "CW Jc":"Cleator & Workington Junction.",
    "DA":"Dundee & Arbroath (original and later Joint).",
    "DB Jt":"Dumbarton & Balloch Joint.",
    "Dist":"District Line (strictly Metropolitan District).",
    "Dock":"Docklands Light.",
    "DPA":"Dundee & Perth & Aberdeen Junction.",
    "EA":"East Anglian.",
    "EC":"Eastern Counties.",
    "Ed & Dalk":"Edinburgh & Dalkeith.",
    "EG":"Edinburgh & Glasgow.",
    "EK":"East Kent Light.",
    "EU":"Eastern Union.",
    "EWYU":"East & West Yorkshire Union.",
    "Fur":"Furness.",
    "FYN":"Freshwater, Yarmouth & Newport.",
    "GBK Jt":"Glasgow, Barrhead & Kilmarnock Joint.",
    "GC":"Great Central.",
    "GC GI":"Great Central (Grimsby & Immingham electric tramway).",
    "GE":"Great Eastern.",
    "GJ":"Grand Junction. ",
    "Glyn":"Glyn Valley Tramway. ",
    "GN":"Great Northern.",
    "GNS":"Great North of Scotland.",
    "GP Jt":"Glasgow & Paisley Joint.",
    "GSW":"Glasgow & South Western.",
    "GU":"Glasgow Underground.",
    "GW":"Great Western. ",
    "HB":"Hull & Barnsley.",
    "HC":"Hammersmith & City Joint.",
    "High":"Highland.",
    "IoW":"Isle of Wight.",
    "IWC":"Isle of Wight Central.",
    "Jub":"Jubilee Line.",
    "KB":"Kilsyth & Bonnybridge Joint.",
    "KE":"Knot(t) End.",
    "KES":"Kent & East Sussex Light.",
    "L&B":"London & Birmingham.",
    "LBSC":"London, Brighton & South Coast.",
    "LCD":"London Chatham & Dover. ",
    "LM":"Liverpool & Manchester.",
    "LMS":"London, Midland & Scottish.",
    "LNE":"London & North Eastern.",
    "LNW":"London & North Western.",
    "LO":"Liverpool Overhead.",
    "LPJ":"Lancaster & Preston Junction.",
    "LPTB":"London Passenger Transport Board.",
    "LSW":"London & South Western.",
    "LTS":"London, Tilbury & Southend.",
    "LU":"Lancashire Union.",
    "LY":"Lancashire & Yorkshire.",
    "Lynton ":"Lynton & Barnstaple. ",
    "Manch":"Manchester Metrolink.",
    "MC":"Maryport & Carlisle.",
    "Met":"Metropolitan Railway/ Line",
    "Met GNC":"Metropolitan (Great Northern & City Section).",
    "MGN":"Midland and Great Northern Joint line. ",
    "Mid":"Midland.",
    "MK":"Monkland & Kirkintilloch.",
    "MS&L":"Manchester, Sheffield & Lincolnshire.",
    "MSJA":"Manchester, South Junction & Altrincham.",
    "MSWJ":"Midland & South Western Junction.",
    "N&B":"Neath & Brecon.",
    "NB":"North British.",
    "NC":"Newcastle & Carlisle.",
    "NE":"North Eastern.",
    "Newtyle":"Dundee & Newtyle.",
    "Nidd":"Nidd Valley Light.",
    "NL":"North London.",
    "Nor":"Northern Line.",
    "Norfolk & S":"Norfolk & Suffolk Joint.",
    "NS":"North Staffordshire.",
    "NSWJ":"North & South Western Junction Joint.",
    "NU":"North Union/North Union Joint.",
    "NWNG":"North Wales Narrow Gauge.",
    "OAGB":"Oldham, Ashton & Guide Bridge Junction Joint. ",
    "PDSW":"Plymouth, Devonport & South Western Junction.",
    "Picc":"Piccadilly Line.",
    "PLA":"Port of London Authority.",
    "PPW Jt":"Portpatrick & Wigtownshire Joint.",
    "PT":"Port Talbot Railway & Docks.",
    "PW":"Preston & Wyre (original and later Joint).",
    "Raven":"Ravenglass & Eskdale.",
    "RHD":"Romney, Hythe & Dymchurch. ",
    "Rhy":"Rhymney.",
    "RSB":"Rhondda & Swansea Bay.",
    "Rye & C":"Rye & Camber Tramway.",
    "S&D":"Stockton & Darlington.",
    "Scot Cent":"Scottish Central.",
    "SD Jt":"Somerset & Dorset Joint.",
    "SE":"South Eastern.",
    "SEC":"South Eastern & Chatham.",
    "SH Jt":"Shrewsbury & Hereford Joint.",
    "SIT":"Swansea Improvements & Tramways Company (Swansea & Mumbles).",
    "SK":"Swinton & Knottingley Joint.",
    "SM":"Shropshire & Montgomeryshire Light.",
    "SMJ":"Stratford-upon-Avon & Midland Junction. ",
    "SR":"Southern.",
    "SSMWC":"South Shields, Marsden & Whitburn Colliery.",
    "SW Jt":"Severn & Wye Joint.",
    "SY":"South Yorkshire (later part of GC).",
    "TFG Jt":"Tottenham & Forest Gate Joint. ",
    "TH Jt":"Tottenham & Hampstead Joint.",
    "TV":"Taff Vale.",
    "TWM":"Tyne & Wear Metro.",
    "Vic":"Victoria Line.",
    "VoR":"Vale of Rheidol.",
    "W Lancs":"West Lancashire.",
    "WCE Jt":"Whitehaven, Cleator & Egremont Joint.",
    "WCP":"Weston, Clevedon & Portishead Light.",
    "WELCP":"West End of London & Crystal Palace.",
    "WH":"Welsh Highland.",
    "WL":"West London Joint (including Extension). ",
    "WMC":"Wilsontown, Morningside & Coltness.",
    "WMCQ ":"Wrexham, Mold & Connah’s Quay.",
    "WP Jt":"Weymouth & Portland Joint.",
    "WRG Jt":"West Riding & Grimsby Joint.",
    "WS":"West Sussex (Selsey Tramway).",
    "WSC Jt":"Woodside & South Croydon Jt.",
    "WSM":"West Somerset Mineral"}

# From Quicks intro:
stntypes = ["AOT", "CLO", "CO", "CO N", "CO TT", "ND", "NG", "NON-TT", "OP", "P", "REOP", "TT", "HL", "LL", "HB", "HBA", "NG", "TT"]

# Most common last tokens:
keywords = ["PLATFORM", "PLATFORMS", "HALT", "INTERNATIONAL", "JUNCTION", "CAMP", "CENTRAL", 'ROAD', 
            'JUNCTION', 'BRIDGE', 'STREET', 'PARK', 'LANE', 'HILL', 'COLLIERY', 'TOWN', 'GREEN', 
            'CENTRAL', 'CROSSING', 'NORTH', 'LEVEL', 'EAST', 'DOCK', 'WEST', 'GATE', 'CROSS', 'HALT', 
            'SOUTH', 'MILL', 'END', 'SIDING', 'HALL', 'HOUSE']

In [6]:
def process_decription(i, mainst, description):
    print(mainst)
    
    # Original formatting error such as "DYKEBAR [", "Cal]"
    if mainst.endswith("["):
        mainst = mainst[:-1]
        description[0] = "[" + description[0]
    
    dSubstations = dict()
    
    rsubst = r"[A-Z ?\-?\&? ?]+ "
    rsubstInitial = r"^(" + mainst[0] + "[ |\-]([A-Z ?\-?\&? ?]+)+) "
    
    substationId = 0
    substname = ""
    
    for line in description:
        match1 = re.match(rsubst, line)
        match2 = re.match(rsubstInitial, line)
        if match2:
            if not match2.group(0).strip() in companies:
                substname = match2.group(0).strip()
                substationId += 1
        elif match1:
            if len(match1.group(0).strip()) > 1 and not match1.group(0).strip() in companies:
                substname = match1.group(0).strip()
                substationId += 1
        if substationId == 0 and substname == "":
            substname = mainst
        stup = (substationId, substname)
        if not stup in dSubstations:
            line = re.sub(r"^" + substname, "", line)
            dSubstations[stup] = line.strip()
        else:
            dSubstations[stup] += " " + line.strip()
                
    return dSubstations

In [7]:
quickdf = pd.read_pickle("quick_processed.pkl")

cols = ['MainId', 'MainStation', 'SubId', 'SubStation', 'Description']
lst = []
for i, row in quickdf.iterrows():
    main_station = row["Station"]
    description = row["Description"]
    dSubstations = process_decription(i, main_station, description)
    for ss in dSubstations:
        lst.append([i, main_station, ss[0], ss[1], dSubstations[ss]])
subsdf = pd.DataFrame(lst, columns=cols)
subsdf.to_csv("substs.tsv", sep="\t")

ABBEY
ABBEY & WEST DEREHAM 
ABBEY FOREGATE 
ABBEY HOUSES 
ABBEY MILLS 
ABBEY OF DEER
ABBEY ROAD
ABBEY WOOD
ABBEY WOOD
ABBEYDORE 
ABBEYHILL 
ABBEYHOLME
ABBOTS RIPTON
ABBOTS WOOD JUNCTION
ABBOTSBURY
ABBOTSCLIFFE BOX
ABBOTSFORD FERRY
ABBOTSHAM ROAD 
ABDIE
ABER
ABER
ABER
ABERAMAN
ABERANGELL 
ABERAVON
ABERAYRON
ABERBARGOED
ABERBEEG
ABERBRAN 
ABERCAIRNY 
ABERCAMLAIS
ABERCANAID 
ABERCARN 
ABERCHALDER 
ABERCORN
ABERCRAVE 
ABERCWMBOI 
ABERCYNON
ABERDARE
ABERDARE JUNCTION 
ABERDEEN
ABERDERFYN 
ABERDOUR 
ABERDOVEY
ABERDYLAIS 
ABEREDW
ABERERCH 
ABERFAN 
ABERFELDY 
ABERFFRWD
ABERFORD
ABERFOYLE 
ABERGAVENNY
ABERGELE & PENSARN 
ABERGLASLYN 
ABERGWILI 
ABERGWYNFI 
ABERGYNOLWYN
ABERKENFIG
ABERLADY 
ABERLLEFENI
ABERLOUR 
ABERMULE
ABERNANT
ABERNETHY
ABERNETHY
ABERSYCHAN
ABERTAFOL
ABERTHAW 
ABERTHIN 
ABERTILLERY
ABERTRIDWR
ABERTYSSWG
ABERYSTRUTH
ABERYSTWYTH
ABINGDON
ABINGDON JUNCTION
ABINGDON ROAD
ABINGDON ROAD
ABINGTON
ABINGTON 
ABOYNE
ABOYNE CURLING POND
ABRAM COLLIERY
ABY
ACCRINGTON
ACH-NA-CLOICH
ACHAN

BOSSALL
BOSTON
BOSTON LODGE
BOSTON MANOR
BOTANIC GARDENS
BOTANIC GARDENS
BOTHLIN VIADUCT
BOTHWELL
BOTLEY
BOTOLPHS BRIDGE
BOTRIPHNIE
BOTT LANE
BOTTESFORD
BOTTISHAM & LODE
BOTTOM OF SUTTON INCLINE
BOTTOM OF WHISTON INCLINE
BOUGHROOD & LLYSWEN
BOUGHTON
BOUGHTON
BOUNDS GREEN
BOURNE
BOURNE BRIDGE
BOURNE END
BOURNEMOUTH
BOURNVILLE
BOURNVILLE MON
BOURTON
BOURTON-ON-THE-WATER
BOVEY
BOVINGTON CAMP
BOW 
BOW
BOW ROAD
BOW BRICKHILL
BOW STREET
BOWBRIDGE CROSSING
BOWDON
BOWER
BOWERS
BOWES
BOWES BRIDGE
BOWES PARK
BOWESFIELD JUNCTION/LANE
BOWHOUSE
BOWKER VALE
BOWLAND
BOWLING
BOWLING
BOWMANHILL BALLAST
BOWNESS
BOWOOD
BOWSHANK
BOX
BOXFORD
BOXHILL
BOXMOOR
BOYCES BRIDGE
BOYNE HILL
BRACEBOROUGH SPA
BRACKENHILLS
BRACKLEY
BRACKNELL
BRADBURY
BRADFIELD
BRADFORD
BRADFORD
BRADFORD PEVERELL & STRATTON
BRADFORD-ON-AVON
BRADING
BRADLEY
BRADLEY & MOXLEY
BRADLEY FOLD
BRADLEY GATE 
BRADNOP
BRADSHAW LEACH
BRADWELL
BRAEMORE
BRAESIDE
BRAFFERTON
BRAIDWOOD
BRAINTREE 
BRAITHWAITE
BRAMBER
BRAMBLEDOWN
BRAMCOTE
BRAMFORD
BRAMHA

CROSTON
CROUCH END
CROUCH HILL
CROW PARK
CROW PARK
CROW ROAD
CROWBOROUGH
CROWCOMBE
CROWDEN
CROWHURST
CROWLAND
CROWLANDS
CROWLE
CROWN POINT
CROWN STREET
CROWTHORNE
CROXALL
CROXDALE
CROXLEY
CROY
CROYDON
CRUCKTON
CRUDEN BAY
CRUDGINGTON
CRUMLIN
CRUMLIN BURROWS
CRUMSTANE
CRYMMYCH ARMS
CRYNANT
CRYSTAL PALACE
CRYTHAN
CUDDINGTON
CUDWORTH
CUERDLEY
CUFFLEY
CUILHILL
CULCHETH
CULGAITH
CULHAM
CULKERTON
CULLEN
CULLERCOATS
CULLINGWORTH
CULLODEN
CULLOMPTON
CULMSTOCK
CULRAIN
CULROSS
CULTER
CULTS
CULWORTH
CUMBERLAND STREET
CUMBERNAULD
CUMBERNAULD ROAD
CUMMERSDALE
CUMMERTREES
CUMMINGSTON
CUMNOCK
CUMWHINTON
CUNNINGHAMHEAD
CUPAR
CURLING POND
CURRIE
CURRIEHILL
CURRY MARSH
CURTHWAITE
CUSTOM HOUSE 
CUSTOM HOUSE
CUTHLIE
CUTLERS GREEN
CUTNALL GREEN
CUTSYKE
CUTTY SARK
CUXTON
CWM
CWM BARGOED
CWM BLAWD
CWM BRIDGE – 
CWM CLYDACH
CWM CYMMER
CWM PRYSOR
CWM-Y-GLO
CWMAMAN 
CWMAVON
CWMAVON MON 
CWMBACH
CWMBRAN
CWMCARN
CWMDU
CWMFFRWD 
CWMFFRWDOER
CWMGORSE 
CWMLLYNFELL
CWMMAWR
CWMMERTYCH
CWMNEOL
CWMRHYD-Y-GAU
CWMS
CWMTWRC

GUESTLING
GUESTWICK
GUIDE BRIDGE
GUILDFORD
GUISBOROUGH
GUISBOROUGH LANE
GUISELEY
GULLANE
GUNNERSBURY
GUNNESS & BURRINGHAM
GUNNISLAKE
GUNTON
GUPWORTHY
GUSHETFAULDS
GUTHRIE
GUYHIRNE
GWAELODYWAEN COLLIERY
GWAUN-CAE-GURWEN
GWENDRAETH COLLIERY
GWERNYDOMEN
GWERSYLLT
GWINEAR
GWYDDELWERN
GWYS
GYFEILLON
GYPSY LANE
HABROUGH
HACHESTON
HACKBRIDGE
HACKNEY
HADDENHAM
HADDENHAM CAMBS
HADDINGTON
HADDISCOE
HADFIELD
HADHAM
HADLEIGH
HADLEY
HADLEY WOOD
HADLOW ROAD
HADNALL
HADNOCK
HADRIAN ROAD
HAFOD
HAFOD GARREGOG
HAFOD RUFFYDD
HAFOD-Y-LLYN
HAFOD-Y-LLYN
HAFODYRYNYS
HAG FOLD
HAGGERLEASES
HAGGERSTON
HAGLEY
HAGLEY ROAD
HAIGH
HAILES
HAILSHAM
HAINAULT
HAINTON STREET
HAIRMYRES
HALBEATH
HALBERTON
HALE
HALE END
HALEBANK
HALESOWEN
HALESOWEN JUNCTION HALT
HALESWORTH
HALEWOOD
HALFWAY
HALFWAY
HALIFAX
HALKIRK
HALL DENE
HALL GREEN
HALL HILLS 
HALL I’ TH’ WOOD
HALL ROAD
HALLATON
HALLATROW
HALLCRAIG (STREET)
HALLEN
HALLIFORD
HALLILOO
HALLING
HALLINGTON
HALLOON 
HALMEREND
HALSALL
HALSHAW MOOR
HALSTEAD
HALSTEAD
HALTON
HALTON

LODGE
LODGE FARM
LODGE HILL
LOFTHOUSE & OUTWOOD
LOFTHOUSE-IN-NIDDERDALE
LOFTUS
LOGIERIEVE
LOGIN
LONDESBOROUGH
LONDESBOROUGH PARK
LONDESBOROUGH ROAD
LONDON
BISHOPS ROAD
BISHOPSGATE
BLACKFRIARS 
BLACKFRIARS
BLACKFRIARS BRIDGE
BRICKLAYERS ARMS
BROAD STREET
CANNON STREET
CHARING CROSS
CITY THAMESLINK
DEVONSHIRE STREET MILE END
EUSTON
FENCHURCH STREET
HOLBORN VIADUCT
HOLBORN VIADUCT
KINGS CROSS
KINGS CROSS THAMESLINK
LIVERPOOL STREET
LONDON BRIDGE
LUDGATE 
MAIDEN LANE
MARYLEBONE 
MINORIES 
MOORGATE 
NINE ELMS
PADDINGTON
PIMLICO
ST PANCRAS
ST PAULS
SHOREDITCH 
SNOW HILL
VICTORIA
WATERLOO
WATERLOO EAST
LONDON & THAMES HAVEN OIL WHARVES
LONDON BRIDGE
LONDON CITY AIRPORT 
LONDON DOCKS
LONDON FIELDS
LONDON ROAD BRIGHTON
LONDON ROAD GUILDFORD
LONG ASHTON
LONG BUCKBY
LONG CLAWSON & HOSE
LONG EATON
LONG LANE
LONG MARSTON
LONG MARTON
LONG MELFORD
LONG PRESTON
LONG STANTON
LONG SUTTON
LONG SUTTON & PITNEY
LONGANNET POWER STATION
LONGBECK
LONGBENTON
LONGBRIDGE
LONGCLIFFE
LONGCROSS
LONGDON 
LONGDON ROA

PHILORTH
PHILPSTOUN
PICCADILLY CIRCUS
PICKBURN & BRODSWORTH
PICKERING
PICKHILL
PICKHILL
PICKLE BRIDGE
PICTON
PIDDINGTON
PIE LANE
PIEL
PIER HEAD
PIERCEBRIDGE
PIERSHILL
PILL
PILLING
PILMOOR
PILNING
PILOT
PILRIG
PILSLEY
PILTON YARD 
PIMBO LANE
PIMHOLE
PIMLICO
PINCHBECK
PINCHINTHORPE
PINEWOOD
PINGED
PINHOE
PINKHILL
PINMORE
PINNER
PINWHERRY
PINXTON
PIPE GATE
PIRTON
PITCAPLE
PITCROCKNIE
PITFODELS
PITFOUR CURLING
PITLOCHRY
PITLURG
PITMEDDEN
PITMUIES MILL
PITSEA
PITSFORD & BRAMPTON
PITTENWEEM
PITTENZIE
PITTINGTON
PITT’S HEAD
PITTS HILL
PLAIDY
PLAINS
PLAISTOW
PLAISTOW
PLANK LANE
PLANTATION
PLAS
PLAS MARL
PLAS POWER
PLAS-Y-COURT
PLAS-Y-NANT
PLASHETTS
PLATT BRIDGE
PLAWSWORTH
PLEALEY ROAD
PLEAN
PLEASINGTON
PLEASLEY
PLEASURE BEACH
PLECK
PLENMELLER
PLESSEY
PLEX MOSS LANE
PLOCKTON
PLODDER LANE
PLOWDEN
PLUCKLEY
PLUM(B)LEY
PLUMPTON
PLUMPTON
PLUMSTEAD
PLUMTREE
PLYM BRIDGE
PLYMOUTH
PLYMPTON
PLYMSTOCK
POCHIN PITS
POCKLINGTON
POINT PLEASANT
POISON CROSS
POKESDOWN
POLEGATE
POLESWORTH
POLLICOTT
POLLOKSHAWS
P

SOUTHWATER
SOUTHWELL
SOUTHWELL JUNCTION
SOUTHWICK
SOUTHWICK
SOUTHWOLD
SOWERBY BRIDGE
SPA ROAD BERMONDSEY
SPALDING
SPARROWLEE
SPEAN BRIDGE 
SPEECH HOUSE ROAD
SPEEDWELL COLLIERY
SPEEN
SPEETON
SPEKE
SPELBROOK
SPELLOW
SPENCER ROAD
SPENNITHORNE
SPENNYMOOR
SPETCHLEY
SPETISBURY
SPEY BAY
SPEY BRIDGE
SPIERSBRIDGE
SPILSBY
SPINK HILL
SPINKS LANE
SPITAL
SPITAL
SPOFFORTH
SPON LANE
SPONDON
SPOONER ROW
SPORLE
SPRATTON
SPREAD EAGLE
SPRING GROVE
SPRING ROAD
SPRING VALE
SPRINGBURN
SPRINGFIELD
SPRINGFIELD
SPRINGFIELD
SPRINGHEAD
SPRINGSIDE
SPRINGWELL
SPROTBOROUGH
SPROUSTON
SQUIRES GATE
SQUIRRELS HEATH
STACKSTEADS
STADDLETHORPE
STADIUM OF LIGHT
STAFFORD
STAFFORD ROAD
STAG & CASTLE
STAINCLIFFE & BATLEY CARR
STAINCROSS
STAINES
STAINFORTH
STAINLAND & HOLYWELL GREEN
STAINMORE
STAINTON DALE
STAIRFOOT
STAITHES
STALBRIDGE
STALEY & MILLBROOK
STALHAM
STALLINGBOROUGH
STALYBRIDGE
STAMFORD
STAMFORD BRIDGE
STAMFORD BROOK
STAMFORD HILL
STAMMERHAM
STANBRIDGEFORD
STANDISH
STANDON
STANDON BRIDGE
STANE STREET
STANELY
STANFO

UNITED STEEL CO’S WORKMEN’S HALT
UNIVERSITY
UNIVERSITY
UNSTONE
UP EXE
UPHALL
UPHILL
UPHOLLAND
UPLAWMOOR
UPLEATHAM MINES 
UPMINSTER
UPNEY
UPPER BANK
UPPER BATLEY
UPPER BIRSTALL
UPPER BOAT
UPPER BROUGHTON
UPPER CWMBRAN
UPPER GREENHILL
UPPER GREENOCK
UPPER HALLIFORD
UPPER HELENSBURG 
UPPER HOLLOWAY
UPPER LYDBROOK
UPPER PONTNEWYDD
UPPER SOUDLEY
UPPER SYDENHAM
UPPER TYNDRUM
UPPER WARLINGHAM
UPPER WATCHINGWELL 
UPPERBY PLANT & MACHINERY WORKSHOP
UPPERMILL
UPPERTHORPE & KILLAMARSH
UPPINGHAM
UPTON
UPTON & BLEWBURY
UPTON & NORTH ELMSALL
UPTON LOVEL(L) CROSSING
UPTON MAGNA
UPTON PARK
UPTON-BY-CHESTER
UPTON-ON-SEVERN
UPWELL
UPWEY
URALITE
URLAY NOOK
URMSTON
URQUHART
USHAW MOOR
USK
USSELBY
USWORTH
UTTERBY
UTTOXETER
UXBRIDGE
UXBRIDGE ROAD
VALLEY
VALLEYFIELD PLATFORM
VAN
VARTEG
VAUXHALL
VAUXHALL
VELVET HALL
VENN CROSS
VENTNOR
VERNEY JUNCTION
VERWOOD
VIADUCT
VIADUCT COTTAGES
VIADUCT YARD
VICARAGE CROSSING
VICKERS PLATFORM
VICTORIA
VICTORIA
VICTORIA
VICTORIA DOCKS
VICTORIA PARK
VICTORIA PARK
VICTORIA R

WYRE DOCK
WYRE FOREST
WYRLEY & CHESLYN HAY
WYTHALL
YALDING
YAPHAM GATE
YAPTON
YARBRIDGE
YARDE
YARDLEY WOOD
YARM
YARMOUTH
YARMOUTH
YARNTON
YATE
YATTON
YAXHAM
YAXLEY
YAXLEY & FARCET
YEADON
YEALAND
YEALMPTON
YEARSETT
YEATHOUSE
YELDHAM
YELVERTOFT & STANFORD PARK
YELVERTON
YEO MILL
YEOFORD
YEOVENEY
YEOVIL
YETMINSTER
YNISCEDWYN
YNYS
YNYS-Y-LLYN
YNYSDDU
YNYSFERLAS 
YNYSFOR 
YNYSHIR
YNYSLAS
YNYSWEN
YNYSYBWL
YNYSYGEINON
YNYSYGEINON JUNCTION
YOCKLETON
YOKER
YORK
YORK & NEWCASTLE JUNCTION
YORK JUNCTION
YORK ROAD
YORK ROAD
YORKHILL
YORTON
YSTALYFERA
YSTRAD CARDIGAN
YSTRAD MYNACH
YSTRAD RHONDDA
YSTRADGYNLAIS 
YSTRADOWEN


### Renaming abbreviated substations

In [8]:
def subst_rename(main, sub):
    sub = sub.replace("&", " AND ")
    sub = re.sub(' +', ' ', sub)
    rsub = []
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
    main = main.translate(translator).split()
    sub = sub.translate(translator).split()
    if sub != main:
        
        # Sometimes, first token is split by whitespace. Join split tokens:
        # e.g. ['F', 'ISHPONDS']
        if len(sub) == 2:
            if sub[0] + sub[1] == main[0]:
                sub = [sub[0] + sub[1]]
        # e.g. ['L', 'ITTLE', 'ORMESBY']
        if len(sub) > 2:
            if sub[0] + sub[1] == main[0]:
                sub = [sub[0] + sub[1]] + sub[2:]
        
        # Sometimes, first token is split by whitespace. Join split tokens:
        # e.g. 'CROSS KEYS' and 'CROSSKEYS'
        if len(main) == 2:
            if main[0] + main[1] == sub[0]:
                main = [main[0] + main[1]]
        # e.g. 
        if len(main) > 2:
            if main[0] + main[1] == sub[0]:
                main = [main[0] + main[1]] + main[2:]
                
        # CASE 1: Main station is just one token, substation is more than one token:
        if len(main) == 1 and len(sub) > 1:
            
            # e.g. 'ALTON' and 'ALTON PARK'
            if main[0] == sub[0]:
                rsub += sub
            
            # e.g. 'BONNYRIGG' and 'BONNYRIGGE DEPOT'
            elif sub[0].startswith(main[0]):
                rsub += sub
            
            # e.g. 'BIRMINGHAM' and 'B NEW STREET'
            elif main[0][0] == sub[0][0] and len(sub[0]) == 1:
                rsub.append(main[0])
                rsub += sub[1:]
            
            # e.g. 'BARGEDDIE' and 'BARGE DDI E'
            elif "".join(sub) == main[0]:
                rsub += main
                
            # e.g. 'ALLOA' and 'SOUTH ALLOA'
            elif any(x == main[0] for x in sub):
                rsub += sub
            
            # e.g. 'AIRDRIE' and 'COMMONHEAD A NORTH'
            elif any(len(x) == 1 and x[0] == main[0][0] for x in sub):
                for x in sub:
                    if len(x) == 1 and x[0] == main[0][0]:
                        rsub.append(main[0])
                    else:
                        rsub.append(x)
                        
            # e.g. 'CAERNARVON' and 'CARNARVON CASTLE'
            elif any(SequenceMatcher(None, x, main[0]).ratio() >= 0.8 for x in sub):
                rsub = sub
                
            # e.g. 'SOUTHPORT' and 'STEAMPORT MUSEUM'
            else:
                rsub.append(main[0])
                rsub += sub
                
        # CASE 2: Substation has length 1, mainstation length > 1
        elif len(sub) == 1 and len(main) > 1:
            
            # e.g. 'CLYDACH ON TAWE' and 'CLYDACH'
            if any(SequenceMatcher(None, x, sub[0]).ratio() >= 0.8 for x in main):
                rsub = sub
                
            # e.g. 'HIGHGATE ROAD' and 'HL'
            elif sub[0] in keywords or sub[0] in stntypes:
                rsub += main
                rsub += sub
                
            # e.g. 'BLAENAU FFESTINIOG' and 'DINAS'
            else:
                rsub = sub
                
        # CASE 3: Substation has length 1, mainstation length 1
        elif len(sub) == 1 and len(main) == 1:
            
            # e.g. 'BELMONT' and 'JUNCTION'
            if sub[0] in keywords or sub[0] in stntypes:
                rsub += main
                rsub += sub
            
            # e.g. 'SELHURST' and 'SELHUST'
            elif SequenceMatcher(None, sub[0], main[0]).ratio() >= 0.8:
                rsub += main
            
            # e.g. 'WALKER' and 'WALKERGATE'
            elif sub[0].startswith(main[0]):
                rsub += sub
                
            # e.g. 'TILBURY' and 'BERTHS'
            else:
                rsub += main
                rsub += sub
        
        # CASE 4: otherwise
        # e.g. 'FINCHLEY ROAD' and 'F R AND FROGNAL'
        # e.g. 'B ON S AND QUORN' and 'BARROW ON SOAR AND QUORN'
        else:
            tempmain = []
            remsub = []
            tsub = []
            for x in sub:
                if len(x) == 1:
                    for m in main:
                        if m.startswith(x) and not m in tsub and not m in remsub:
                            tsub.append(m)
                            remsub.append(x)
                    else:
                        tsub.append(x)
                else:
                    tsub.append(x)
                    
            rsub = [x for x in tsub if not x in remsub]
            
            # e.g. "ST A" or "S C" as substation names:
            if len(max(rsub, key=len)) <= 2:
                rsub = main
                
        rsub = " ".join(rsub)
        if re.search(r"\bLL\b", rsub):
            rsub = re.sub(r"\bLL\b", "LOW LEVEL", rsub)
        if re.search(r"\bHL\b", rsub):
            rsub = re.sub(r"\bHL\b", "HIGH LEVEL", rsub)
            
        return rsub
    else:
        return " ".join(sub)
        
subsdf['SubStFormatted'] = subsdf.apply(lambda row: subst_rename(row["MainStation"], row["SubStation"]), axis = 1)

In [10]:
subsdf[:50]

,MainId,MainStation,SubId,SubStation,Description,SubStFormatted
0,1,ABBEY,1,A TOWN,[NB] op 3 September 1856** as A; TOWN added 18...,ABBEY TOWN
1,1,ABBEY,2,A JUNCTION,[Cal] op 31 August 1870 (co ½ T 26 September)...,ABBEY JUNCTION
2,1,ABBEY,3,A JUNCTION,[NB] op 8 August 1870 (D&C 14) ; clo 1 Septem...,ABBEY JUNCTION
3,2,ABBEY & WEST DEREHAM,0,ABBEY & WEST DEREHAM,[GE] op 1 August 1882 (Thetford & Watton Times...,ABBEY AND WEST DEREHAM
4,3,ABBEY FOREGATE,0,ABBEY FOREGATE,– see SHREWSBURY.,ABBEY FOREGATE
5,4,ABBEY HOUSES,0,ABBEY HOUSES,see BEAUCHIEF .,ABBEY HOUSES
6,5,ABBEY MILLS,0,ABBEY MILLS,pumping station [private line south of West Ha...,ABBEY MILLS
7,6,ABBEY OF DEER,0,ABBEY OF DEER,[GNS] (non-tt): pilgrimages; train for Roman C...,ABBEY OF DEER
8,7,ABBEY ROAD,0,ABBEY ROAD,[Dock] op 31 August 2011 (RM November) ; still...,ABBEY ROAD
9,8,ABBEY WOOD,0,ABBEY WOOD,[SE] op on or by 1 November 1849 (included in ...,ABBEY WOOD
